In [12]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [13]:
from itertools import combinations

# generate random combination with number n
def combine(temp_list, n):
    return list(combinations(temp_list, n))

In [14]:
# set semantics and compute strength of arguments

filename = "../../bags/app.bag"
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG(filename)
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 3.600000000000002
Argument(name=0, weight=0.5, strength=0.4245437375012453)
Argument(name=1, weight=0.5, strength=0.4245437375012453)
Argument(name=2, weight=0.7302785763532248, strength=0.7302785763532248)
Argument(name=3, weight=0.7036425461655201, strength=0.7036425461655201)



0.0009615120122029308

In [15]:
# get all the attacks and supports in QBAF
att_relation = model.BAG.get_attacks()
sup_relation = model.BAG.get_supports()
all_relation = att_relation + sup_relation

In [16]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', '0', '1'), ('att', '1', '0')]

In [17]:
# set topic argument
topic_a = "1"
topic_arg = model.BAG.arguments[topic_a]

In [18]:
# compute strength for all arguments
for arg in model.BAG.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('0', 0.5, 0.4245437375012453)
('1', 0.5, 0.4245437375012453)
('2', 0.7302785763532248, 0.7302785763532248)
('3', 0.7036425461655201, 0.7036425461655201)


In [19]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    # split current and the rest relation
    cur_relation = all_relation_new[i]
    rest_relation = all_relation_new.copy()
    rest_relation.remove(cur_relation)

    # compute combinations for the rest_relation
    combine_rest_relation = []
    for j in range(len(rest_relation)+1):
        combine_rest_relation.extend(combine(rest_relation, j))

    difference = []
    coefficient = []

    # remove relations in combine_rest_relation
    for j in combine_rest_relation:
        for k in j:
            # remove att
            if k[0]=="att":
                temp_attacker = model.BAG.remove_attack(k[1],k[2]) # k[1] attacks k[2]
            # remove sup
            elif k[0]=="sup":
                temp_supporter = model.BAG.remove_support(k[1],k[2]) # k[1] supports k[2]

        # compute sigma(S U {i})
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength

        # remove cur_relation
        if cur_relation[0]=="att":
            temp_attacker = model.BAG.remove_attack(cur_relation[1],cur_relation[2])
        elif cur_relation[0]=="sup":
            temp_supporter = model.BAG.remove_support(cur_relation[1],cur_relation[2])

        # compute sigma(S)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)

        # compute the coefficient
        S = len(j)
        coefficient.append(math.factorial(n-1-S) * math.factorial(S) / math.factorial(n))

        # recover QBAF for the next iteration
        model = grad.semantics.QuadraticEnergyModel()
        model.approximator = grad.algorithms.RK4(model)
        model.BAG = grad.BAG(filename)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    # compute attributions
    attribution[cur_relation] = sum(np.multiply(coefficient, difference))

QuadraticEnergyModel, RK4
Time: 0.1
Argument(name=0, weight=0.5, strength=0.4244593478772404)
Argument(name=1, weight=0.5, strength=0.4244593478772404)
Argument(name=2, weight=0.7302785763532248, strength=0.7302785763532248)
Argument(name=3, weight=0.7036425461655201, strength=0.7036425461655201)

QuadraticEnergyModel, RK4
Time: 4.799999999999999
Argument(name=0, weight=0.5, strength=0.4011526534994821)
Argument(name=1, weight=0.5, strength=0.49937831683801126)
Argument(name=2, weight=0.7302785763532248, strength=0.7302785763532248)
Argument(name=3, weight=0.7036425461655201, strength=0.7036425461655201)

QuadraticEnergyModel, RK4
Time: 3.5299999999999687
Argument(name=0, weight=0.5, strength=0.4246097055317115)
Argument(name=1, weight=0.5, strength=0.4246097055317115)
Argument(name=2, weight=0.7302785763532248, strength=0.7302785763532248)
Argument(name=3, weight=0.7036425461655201, strength=0.7036425461655201)

QuadraticEnergyModel, RK4
Time: 4.799999999999999
Argument(name=0, weight

In [20]:
attribution

{('att', '0', '1'): -0.08643395487097505,
 ('att', '1', '0'): 0.011375019949063703}